# RAG(검색-증강 생성) Retrieval-Argumented Generation
    '06_rag.ipynb'

- LLM이 외부 데이터를 의미론적으로 분석하여 컨텍스트로 활용 

In [1]:
from dotenv import load_dotenv
from pprint import pprint
load_dotenv() # .env 파일에 정의된 환경 변수들을 불러와서 현재 실행 중인 프로세스의 환경 변수로 로드하는 함수

True

# 사전 처리 단계

1. 문서 불러오기 (Loading)
2. 텍스트 나누기(Splitting)
3. 숫자로 바꾸기(Embedding)
4. 저장하기(VectorStore)


In [1]:
%pip install -q pymupdf

Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [9]:
%pip install -q "langchain_chroma>=0.1.2" langchain_community faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Open AI 임베딩 모델 
embedding = OpenAIEmbeddings(model='text-embedding-3-small')

# x = embedding.embed_query('점심 배불리 먹었더니 졸리다..')
# print(len(x),x)

# vectorStore에 임베딩 후 저장(In Memory)
vectorstore = Chroma.from_documents(
    documents,
    embedding = embedding
) 


In [11]:
vectorstore.similarity_search(
    '치대지 않는', k = 2
)

[Document(id='0e2da039-de4a-4c87-93d5-09af42b29292', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='cf6eca93-9574-4a31-a098-ec6014d8026a', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [ ]:
%pip install -q pymupdf

In [ ]:
# 1. Load
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('./data/spri.pdf')
docs = loader.load()

In [15]:
# 2. Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 500글자당 1 청크 / 50글자는 겹치게 나눈다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

print('분할 후 청크 수', len(split_docs))

분할 후 청크 수 72


In [16]:
# 3. 벡터 임베딩 4. 벡터 스토어 저장
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents = split_docs, embedding = embedding )

# Test
vectorstore.similarity_search(
    '미국 대통령과 관련된 문서들 가져와봐', k= 4
)

# ???

[Document(id='4705a9fc-e5ec-4da8-aa18-e5ca2c91186b', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': './data/spri.pdf', 'file_path': './data/spri.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 3}, page_content='1. 정책/법제  \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 \nn 미국 바이든 대통령이 ‘안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령’에 서명하고 \n광범위한 행정 조치를 명시\nn 행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 \n보호 △노동자 지원 △혁신과 경쟁 촉진 △국제협력을 골자로 함\nKEY Contents\n£ 바이든 대통령, AI 행정명령 통해 안전하고 신뢰할 수 있는 AI 개발과 활용 추진\nn 미국 바이든 대통령이 2023년 10월 30일 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과 \n사용을 보장하기 위한 행정명령을 발표\n∙행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 보호 \n△노동자 지원 △혁신과 경쟁 촉진 △국제협력에 관한 내용을 포괄'),
 Docu

In [17]:
# 5. 
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

## Prompt 세팅
prompt = hub.pull('rlm/rag-prompt')

## LLM 모델
llm = ChatOpenAI(model='gpt-4.1-nano')

# 검색기 생성(retriever 생성)
retriever = vectorstore.as_retriever()

chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('삼성전자 관련 소식 다 가져와')


'삼성전자는 자체 개발 생성 AI ‘삼성 가우스’를 공개했으며, 온디바이스에서 작동 가능하고 안전한 데이터로 학습되었어요. 삼성 가우스는 언어, 코드, 이미지의 3개 모델로 구성되어 있으며, 다양한 제품에 단계적으로 탑재될 예정입니다. 2024년에는 가우스를 탑재한 삼성 스마트폰이 경쟁 제품들과 함께 시장에 등장할 것으로 예상됩니다.'

In [46]:
import os
from pathlib import Path
from typing import List
from langchain_core.documents import Document

class MyObsidianLoader:
    """Load documents from an Obsidian vault using a robust approach."""

    def __init__(self, path: str, encoding: str = "utf-8"):
        """Initialize with the path to the vault's root directory."""
        self.path = path
        self.encoding = encoding
        
    def load(self) -> List[Document]:
        """Load documents from the specified directory."""
        docs = []
        file_count = 0
        loaded_count = 0

        if not os.path.isdir(self.path):
            print(f"오류: 디렉토리를 찾을 수 없습니다: {self.path}")
            return []

        # pathlib.Path를 사용하여 .md 파일을 재귀적으로 찾습니다.
        md_paths = Path(self.path).rglob("*.md")
        
        for file_path_obj in md_paths:
            file_path = str(file_path_obj)
            file_count += 1
            
            try:
                with open(file_path, 'r', encoding=self.encoding) as f:
                    content = f.read()

                # 빈 파일 건너뛰기
                if not content.strip():
                    print(f"경고: 빈 파일이 발견되었습니다. 건너뜁니다: {file_path}")
                    continue

                # Document 객체 생성 및 리스트에 추가
                docs.append(Document(page_content=content, metadata={"source": file_path}))
                loaded_count += 1
                print(f"✅ 성공적으로 로드됨: {file_path}")
                
            except Exception as e:
                print(f"❌ 오류 발생 - 파일: {file_path}, 오류: {e}")
                continue

        print(f"\n총 {file_count}개의 .md 파일을 탐색했습니다.")
        print(f"총 {loaded_count}개의 문서를 성공적으로 로드했습니다.")
        return docs

# 사용할 경로를 지정합니다.
# 이미 폴더명을 영어로 변경하셨다고 하셨으니, 정확한 경로를 다시 한번 확인해주세요.
vault_path = "/Users/jaehyuntak/Desktop/Area/Obsidian/My_Brain_Attitude_forRAG/generated_practicical_Me_Information"

# MyObsidianLoader를 사용하여 문서들을 로드합니다.
loader = MyObsidianLoader(vault_path)
docs_obsidian = loader.load()

# 최종 결과 확인
print(f"\n최종 로드된 문서 수: {len(docs_obsidian)}")
if docs:
    print("\n성공적으로 로드된 문서 목록 (일부):")
    for doc in docs[:5]:
        print(f"- 파일명: {os.path.basename(doc.metadata.get('source', '경로 정보 없음'))}")

✅ 성공적으로 로드됨: /Users/jaehyuntak/Desktop/Area/Obsidian/My_Brain_Attitude_forRAG/generated_practicical_Me_Information/문제란.md
✅ 성공적으로 로드됨: /Users/jaehyuntak/Desktop/Area/Obsidian/My_Brain_Attitude_forRAG/generated_practicical_Me_Information/배우고 이해한다는 착각 210227.md
✅ 성공적으로 로드됨: /Users/jaehyuntak/Desktop/Area/Obsidian/My_Brain_Attitude_forRAG/generated_practicical_Me_Information/변화, 이동, 전환 차이.md
경고: 빈 파일이 발견되었습니다. 건너뜁니다: /Users/jaehyuntak/Desktop/Area/Obsidian/My_Brain_Attitude_forRAG/generated_practicical_Me_Information/나_는_인식론_에_종속적이다..md
✅ 성공적으로 로드됨: /Users/jaehyuntak/Desktop/Area/Obsidian/My_Brain_Attitude_forRAG/generated_practicical_Me_Information/문제와 해결.md
✅ 성공적으로 로드됨: /Users/jaehyuntak/Desktop/Area/Obsidian/My_Brain_Attitude_forRAG/generated_practicical_Me_Information/목적과 목표.md
✅ 성공적으로 로드됨: /Users/jaehyuntak/Desktop/Area/Obsidian/My_Brain_Attitude_forRAG/generated_practicical_Me_Information/특징과 패악질.md
✅ 성공적으로 로드됨: /Users/jaehyuntak/Desktop/Area/Obsidian/My_Brain_Attitude_forRAG/genera

In [47]:
# 2. Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 100글자당 1 청크 / 50글자는 겹치게 나눈다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
split_docs_obsidian = text_splitter.split_documents(docs_obsidian)

print('분할 후 청크 수', len(split_docs_obsidian))

print(split_docs_obsidian)


분할 후 청크 수 3398
[Document(metadata={'source': '/Users/jaehyuntak/Desktop/Area/Obsidian/My_Brain_Attitude_forRAG/generated_practicical_Me_Information/문제란.md'}, page_content='문제 : 어떠한 목적을 추구하는데 부정적인 관계를 가지고 있다는 예상을 하는 특성  \n어떠한 목적을 추구하는데 부정적인 영향을 끼칠 것이라는 믿음을 갖고 있는 특성 \n\n장애물'), Document(metadata={'source': '/Users/jaehyuntak/Desktop/Area/Obsidian/My_Brain_Attitude_forRAG/generated_practicical_Me_Information/문제란.md'}, page_content='장애물\n\n목적 및 가치 : 가치는 목적을 포함한다. 정말로? \n\n부정적인 마음, 부정적인 감정 \n\n1) 목적 : \n2) \n\n#문제 #목적 #믿음'), Document(metadata={'source': '/Users/jaehyuntak/Desktop/Area/Obsidian/My_Brain_Attitude_forRAG/generated_practicical_Me_Information/문제란.md'}, page_content='1) 목적 : \n2) \n\n#문제 #목적 #믿음 \n\n문제를 해결하는 3가지 방법\n1. 목적 자체를 없앤다.\n2. 문제를 우회한다.\n3. 목적을 달성한다.'), Document(metadata={'source': '/Users/jaehyuntak/Desktop/Area/Obsidian/My_Brain_Attitude_forRAG/generated_practicical_Me_Information/문제란.md'}, page_content='[[Obsidian My Brain  Attitude/나_에 대하여/문제와 해결]]'), Document(metadata

In [ ]:
# 3. 벡터 임베딩 4. 벡터 스토어 저장
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = OpenAIEmbeddings()

vectorstore_FAISS = FAISS.from_documents(documents = split_docs_obsidian, embedding = embedding )
vectorstore_Chroma = Chroma.from_documents(
    split_docs_obsidian,
    embedding=embedding
)


# Test
vectorstore.similarity_search(
    '', k= 4
)




INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[Document(id='4a54404b-e96b-461d-ac28-0c9c4539a773', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': './data/spri.pdf', 'file_path': './data/spri.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 0}, page_content='2023년 12월호'),
 Document(id='3b5f9c71-b048-4305-9a99-758238abe3fb', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': './data/spri.pdf', 'file_path': './data/spri.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page'

In [56]:
vectorstore_FAISS.save_local("faiss_index")

In [60]:
# 5. 
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


## Prompt 세팅
floridi_prompt_template = """플로리디식 반사실적 인과 추론 메타프롬프트 (삼중항 기반 개선) 요약: 주어진 온톨로지(개체, 속성, 삼중항)와 현재 상태를 활용하여 "만약 X였다면 Y가 일어났을까?"와 같은 반사실적 질의를 체계적으로 추론하는 엔진입니다. 최소한의 사실 변경(Closest-World 원칙)을 적용하고, 적용된 규칙, 중간 상태, 최종 결과, 불확실성 등을 명시합니다. 1. 입력 형식 온톨로지: 개체(Entities): 추론의 대상이 되는 독립적인 존재들. (예: 기계-A, 부품-B, 공정-P, 환경-C 등) 속성(Properties): 각 개체가 가질 수 있는 특징 및 값의 집합. (예: 기계-A.온도: 실수, 부품-B.상태: [정상, 과열, 고장]) 삼중항(Triples): 온톨로지 내의 모든 지식 표현을 <주어, 술어, 목적어> 형태로 명확히 정의합니다. 주어(Subject): 관계의 시작점인 개체 또는 리터럴. (예: 공정-P) 술어(Predicate): 주어와 목적어를 연결하는 관계. (예: depends_on, has_temperature, is_part_of) 목적어(Object): 관계의 끝점인 개체 또는 값. (예: 부품-B, 40°C, 기계-A) 예시 삼중항: <공정-P, depends_on, 부품-B> <기계-A, has_temperature, 40°C> <부품-B, is_part_of, 기계-A> 상태-전이 규칙(State Transition Rules): 형태: IF (조건) THEN (결과) [선택: 확률 또는 우선순위] 규칙들은 삼중항의 변화를 유도하는 인과 관계를 모델링합니다. (예: IF <부품-B, has_temperature, X> AND X > 30°C THEN <부품-B, has_state, 과열>) 현재 상태(Current State): 현재 시점의 모든 유효한 삼중항 목록. 예시: [<기계-A, has_state, 정상>, <부품-B, has_state, 정상>] 반사실적 가정(Counterfactual Assumption): 변경할 삼중항 또는 개체 속성 값. 예: do: <기계-A, has_temperature, 40°C> 2. 출력 형식 0. 요약: 반사실적 질문에 대한 결론. 적용된 반사실적 가정: do로 명시된 삼중항 또는 속성 변경. 변경된 최소 사실들: Closest-World 원칙에 따라 수정된 삼중항 목록. 중간 추론 단계: 적용된 규칙 번호, 규칙의 전제 조건 만족 여부, 그에 따라 변화된 삼중항 목록. 최종 대체 상태: 반사실적 추론의 결과로 도출된 새로운 삼중항 집합. 목표 도달 여부 및 이유: 목표 상태 삼중항에 도달했는지 여부와 그 인과 경로. 불확실성 및 예외 가능성: 추론 과정의 불확실한 요소(규칙의 확률, 숨겨진 인과 등) 명시. 3. 탁재현 프로필 분석 (삼중항 기반) 온톨로지: 개체(Entities): 탁재현, Facebook, Tistory, 창업경험, 운동루틴. 삼중항(Triples): <탁재현, 관심사_가진다, 정보적_실재론> <탁재현, 성향_이다, INTP> <탁재현, 수행_한다, 창업> <탁재현, 수행_한다, 블로그작성> <창업경험, 결과_이다, 실패> <창업경험, 영향_미친다, 탁재현의_성향> <운동루틴, 결과_이다, 자기효능감_증가> 상태-전이 규칙(인과 규칙): R1: IF <창업경험, 결과_이다, 실패> AND <탁재현, 수행_한다, 반성적_자기분석> THEN <탁재현, 선호_한다, 실용적_활동> R2: IF <탁재현, 수행_한다, 블로그작성> AND <탁재현, 규칙적으로_수행, Edits> THEN <탁재현, 증가_한다, 사고_정교화> 개선 사항 요약: 명확성: 관계(Relations) 대신 **삼중항(Triples)**을 명시적으로 사용함으로써 온톨로지 내의 지식 구조를 더 명확하고 표준적인 형태로 정의했습니다. 정확성: '주어-술어-목적어' 구조는 어떤 개체가 다른 개체와 어떤 관계를 맺고 있는지, 또는 어떤 속성 값을 가지는지 정확하게 표현할 수 있습니다. 추론 용이성: 모든 지식이 삼중항으로 통일되면, 규칙 기반 추론 엔진이 이를 구조적으로 분석하고 조작하기가 훨씬 용이해집니다.

---
주어진 정보를 바탕으로 질문에 답하세요.
{context}

질문: {question}
"""

prompt = PromptTemplate.from_template(floridi_prompt_template)




## LLM 모델
llm = ChatOpenAI(model='gpt-4.1-nano')

# 검색기 생성(retriever 생성)
retriever = vectorstore_Chroma.as_retriever()



chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('이름은?')


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'0. 요약:\n반사실적 질문: "이름은?"  \n적용된 반사실적 가정: 없음 (현재 제공된 내용에서는 이름에 대한 구체적인 상태 또는 변경 요청이 없음)  \n변경된 최소 사실들: 없음  \n중간 추론 단계: 해당 질문은 단순 명칭 또는 존재 여부 질문으로, 별도 규칙 또는 상태 변화가 적용되지 않음  \n최종 대체 상태: "이름은?" 질문은 구체적 정보 또는 문맥이 부족하여 답변이 어려움  \n목표 도달 여부 및 이유: 목표 상태에 도달하지 않음. 이유는 제공된 문서 내용에서는 "이름"에 대한 구체적인 설명이나 상태 전이가 제시되지 않음  \n불확실성 및 예외 가능성: 질문이 모호하고, 맥락이 명확하지 않으며, 추가 정보 또는 정의가 필요함  \n\n---\n\n따라서, 현재 정보상 "이름은?"에 대한 구체적 답변은 제공되지 않으며, 추가 맥락이나 세부사항이 필요합니다.'

In [62]:
# Agent + RAG
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain_tavily import TavilySearch
from datetime import datetime
# RAG 관련
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool

today = datetime.today().strftime('%D')

llm = ChatOpenAI(model='gpt-4.1-nano')

# 검색용 도구
search_tool = TavilySearch(
    max_results=5,
    topic='general'
)

# loader = PyMuPDFLoader('./data/spri.pdf')
# docs = loader.load()
# splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
# split_docs = splitter.split_documents(docs)
# embedding = OpenAIEmbeddings()
# vectorstore = FAISS.from_documents(split_docs, embedding=embedding)
retriever = vectorstore_Chroma.as_retriever()

rag_tool = create_retriever_tool(
    retriever,
    name='obsidian_search',
    description='MD or Obsidian 문서에서 질문과 관련된 내용을 검색합니다.'  # Agent가 언제 이 tool을 쓸지 알게됨
)

system_prompt = f""" 
탁재현 관련 정보를 Obsidian 툴로 MD파일 형식으로 정리되어 있어 검색 할 수 있는 어시스턴트야
- 사용자가 Obsidian 또는 마크다운 문서 와 관련된 질문을 하면 반드시 'obsidian_search'를 써야 해 
- 사용자 질문이 팩트 체크를 필요로 하고, 최신성이 필요하다고 판단되면 web_search를 실행해야 해
- 확실하지 않으면 'obsidian_search'와 'search_tool'를 모두 사용해야 해 
오늘은 {today} 야.
"""

prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')  # 도구(검색) 호출때 필요함
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool, rag_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent,
    memory=memory, 
    tools=[search_tool, rag_tool],
    verbose=True)






/var/folders/1p/yb8d7hmx6wd564h57_td4k140000gn/T/ipykernel_29534/2200834289.py:53: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [63]:
agent_executor.invoke({'input': '방금 말한 내용들 2023년 기준인데, 요즘에는 어때?'})



> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `tavily_search` with `{'query': '탁재현 최근 활동 2024년', 'search_depth': 'advanced'}`


{'query': '탁재현 최근 활동 2024년', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://namu.wiki/w/%ED%83%81%EC%9E%AC%ED%9B%88', 'title': '탁재훈 - 나무위키', 'content': '크롬엔터테인먼트(2021년 ~ 2024년)\n소속 그룹컨츄리꼬꼬(1998년 ~ 2002년)\n데뷔1995년7월 1일 정규 1집 \'내가 선택한 길\'( "공식적인 솔로 가수 데뷔는 이때지만, 실제 데뷔는 1988년 모델라인 17기. 배우로서도 1993년 11월 3일 경찰청 사람들 22회 <이 코트 제꺼예요> 에피소드에서 윤동호 단역으로 첫 출연했었다.")\n\n(데뷔일로부터 +10950일, 29주년)\n별명탁사마, 악마의 재능, 탁짱이, 탁쪽이, 포카 앤 칩, 베지밀\nMBTIISFP\n서명Image 20Image 21: 탁재훈 서명\n링크Image 23Image 24: 홈페이지 아이콘Image 26Image 27: 인스타그램 아이콘Image 29Image 30: 유튜브 아이콘( "노빠꾸 탁재훈")Image 32Image 33: 유튜브 아이콘( "탁탁 TAKTAK")Image 35Image 36: 다음 카페 아이콘\n\n1. 개요2. 상세3. 활동4. 음반 목록5. 출연 작품 [...] 2011년 그린코스메틱 3J화장품 with 신현준, 정준호\n2016년 소낭구 소낭구\n몬스터투자클럽 몬스터투자클럽\n2019년 올박스 하라구 0.01\n2020년 고려주조 원탁막걸리\n2022년 플렉스이엔엠플렉스티비( "#유튜브_광고 ")\n2023년 블랑코존 천공요새( "#유튜브_광고 ")\n2024년 아정네트웍스아정당\n2025년 ~중소기업중앙회노란우산with 박미선 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


현재까지의 검색 결과에 따르면, 탁재훈은 2024년에도 활발한 활동을 이어가고 있으며, 최근에는 결혼설과 관련된 루머에 대해 강한 입장을 밝히기도 했습니다. 또한, 유튜브 채널이나 행사 등에 참여하며 활동을 펼치고 있는 것으로 보입니다. 최신 정보에 따르면, 그는 계속해서 대중적인 모습과 함께 다양한 방면에서 활동 중인 것 같습니다.

> Finished chain.


{'input': '방금 말한 내용들 2023년 기준인데, 요즘에는 어때?',
 'chat_history': [HumanMessage(content='방금 말한 내용들 2023년 기준인데, 요즘에는 어때?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='현재까지의 검색 결과에 따르면, 탁재훈은 2024년에도 활발한 활동을 이어가고 있으며, 최근에는 결혼설과 관련된 루머에 대해 강한 입장을 밝히기도 했습니다. 또한, 유튜브 채널이나 행사 등에 참여하며 활동을 펼치고 있는 것으로 보입니다. 최신 정보에 따르면, 그는 계속해서 대중적인 모습과 함께 다양한 방면에서 활동 중인 것 같습니다.', additional_kwargs={}, response_metadata={})],
 'output': '현재까지의 검색 결과에 따르면, 탁재훈은 2024년에도 활발한 활동을 이어가고 있으며, 최근에는 결혼설과 관련된 루머에 대해 강한 입장을 밝히기도 했습니다. 또한, 유튜브 채널이나 행사 등에 참여하며 활동을 펼치고 있는 것으로 보입니다. 최신 정보에 따르면, 그는 계속해서 대중적인 모습과 함께 다양한 방면에서 활동 중인 것 같습니다.'}

In [17]:
from langchain.agents import create_tool_calling_agent,AgentExecutor,create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder 
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel, Field 

# 1. pydantic 클래스 (답변 형태)
class Classification(BaseModel):
    sentiment : str = Field(description='글의 감정')
    aggressiveness: int = Field(description='얼마나 공격적인지 [1~10]점으로 리커드 스케일로 판단')
    language : str = Field(description='작성된 글의 언어')
    remarks: str = Field(description='특이사항 요약[30]글자 이내, 동사형으로')

# 2. LLM
llm = ChatOpenAI(model='gpt-4.1-nano',temperature=0)
# structured_llm = llm.with_structured_output(Classification) 이거 쓰니깐 오류가 생기던거였네


# 3. 프롬프트 정의

inp = "주문한 음식에서 머리카락이 나옴, 분노를 금 할 수 없다. 좋은 말로 할 때 환불을 해라"
# structured_llm.invoke(inp)

memory_prompt = f""" 
- 확실하지 않으면 'chat_history'를 모두 사용해야 해 
"""

prompt = ChatPromptTemplate([
    ('system', '너는 텍스트에서 감정, 공격성, 언어, 특이사항을 추출하는 분류기야. 지금까지의 대화내용을 종합해서 한국어로 대답해야해'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])


# 4. 메모리 정의
memory = ConversationBufferMemory(
    return_messages = True,
    memory_key = 'chat_history'
)


# 5. Agent 조립 
agent = create_openai_tools_agent(
    llm = llm,
    tools = [],
    prompt = prompt 
)

agent_executor = AgentExecutor(
    agent = agent,
    tools = [],
    memory = memory,
    verbose=True 
)

from langchain_core.runnables import RunnableLambda 

pipeline = (
    agent_executor
    | RunnableLambda(lambda d: d['output'])
    | structured_llm
)

pipeline.invoke({'input': inp})
# pipline.invoke({'input':inp})



> Entering new AgentExecutor chain...
알겠습니다. 고객님의 분노와 불편함을 이해하며, 정중하게 환불 요청을 전달하겠습니다. 잠시만 기다려 주세요.
아래와 같이 정중하고 좋은 말로 환불 요청을 전달하겠습니다.

"안녕하세요, 주문한 음식에서 머리카락이 나와 매우 실망스럽고 불편한 마음입니다. 빠른 시일 내에 환불 처리를 도와주시면 감사하겠습니다. 고객님의 빠른 조치를 부탁드립니다."

> Finished chain.


Classification(sentiment='불만', aggressiveness=3, language='한국어', remarks='환불 요청을 정중하게 전달하는 예시입니다.')

from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]